# Geostrophy
## Modules

In [ ]:
using PhysOcean
using DIVAnd
using Statistics
using Makie, CairoMakie, GeoMakie

## Grid and metrics
<div class="alert alert-block alert-warning">
⚠️ Metrics using meters!
</div>

In [ ]:
dx = dy = 0.2

lonr = 27:dx:42
latr = 40:dy:47

depthr = [0.5, 10, 20, 30, 50, 75, 100, 125, 150, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000, 
    1100, 1200, 1300, 1400, 1500, 1750, 2000];

mask, (pm, pn, po), (xi, yi, zi) = DIVAnd_rectdom(lonr, latr, depthr)

Now into meters

In [ ]:
(pmm, pnm) = DIVAnd_metric(xi[:, :, 1], yi[:, :, 1])
pm = repeat(pmm, inner = (1, 1, length(depthr)));
pn = repeat(pnm, inner = (1, 1, length(depthr)));

mask[1:2, :, :] .= false
mask[end-1:end, :, :] .= false;
#mask[:,1:2,:]=false
#mask[:,end-1:end,:]=false

## Observations
We work with fake data.

In [ ]:
temp =
    16 .- zi ./ 1600 .+ cos.(1.4 * xi .+ 0 * xi .- zi / 300) .+
    xi / 5 .* xi ./ (zi .+ 1) / 2000 .* (zi / 1000 .+ xi)
salt = 28 .+ xi

dens = PhysOcean.density.(salt, temp, 0.0) .- 1025;
@info(size(dens));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density field at surface")
hm = heatmap!(ax, xi[:, 1, 1], yi[1, :, 1], dens[:, :, 1])
Colorbar(fig[1, 2], hm, label = "kg/m³")
fig

## Calculate geostrophic velocities 
Assuming level 10 is no motion

In [ ]:
velocities, eta, Vflux = geostrophy(mask, dens, (pm, pn, po), (xi, yi, zi); znomotion = 10);
eta = eta .- mean(eta);

Look at fluxes in y direction (south-north)

In [ ]:
@show mean(var(Vflux[1] .^ 2)), mean(var(Vflux[2] .^ 2));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
hm = heatmap!(ax, xi[:, 1, 1], yi[1, :, 1], eta)
Colorbar(fig[1, 2], hm)
fig

Now analyse `eta` with constrain to have barotropic flow compensating the fluxes just calculated

In [ ]:
# function to interpolate
fun(x, y) = sin.(6x) * cos.(6y)

# observations

x = [35];
y = [42];
f = [0.1]

h = deepestpoint(mask, zi)
len = 500000.2
epsilon2 = 110000
rfluxes = 0.0000001

#Need to scale fluxes by f/g
gsf = (earthgravity.(yi)./coriolisfrequency.(yi))[:, :, 1]
fi, s = DIVAndrun(
    mask[:, :, 1],
    (pm[:, :, 1], pn[:, :, 1]),
    (xi[:, :, 1], yi[:, :, 1]),
    (x, y),
    f,
    len,
    epsilon2;
    moddim = [0, 7],
    topographyforfluxes = (gsf .* h, 0),
    fluxes = (Vflux[1], 0),
    epsfluxes = rfluxes,
    alphabc = 2,
    alpha = [1, 0, 1],
);

Correction term

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
hm = heatmap!(ax, xi[:, 1, 1], yi[1, :, 1], fi)
Colorbar(fig[1, 2], hm)
fig

For plotting and further calculation

In [ ]:
fi[isnan.(fi)] .= 0;

For verification look at how the correction term if applied with overall factor 0 to 2 changes fluxes

In [ ]:
sf = collect(0.0:0.11:2.5)
Ff = zeros(size(sf))
Ff2 = zeros(size(sf))
Vfluxafter = 0
for i = 1:size(sf)[1]

    velafter, etaafter, Vfluxafter =
        geostrophy(mask, dens, (pm, pn, po), (xi, yi, zi); ssh = eta + sf[i] * fi)
    Ff[i] = mean(Vfluxafter[1] .^ 2)
    Ff2[i] = mean(Vfluxafter[2] .^ 2)
end

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Fluxes as a function of amplitude of correction factor")
lines!(ax, sf, Ff)
fig

In [ ]:
@show mean(Vflux[1] .^ 2)

So we can indeed use

In [ ]:
velafter, etaafter, Vfluxafter =
    geostrophy(mask, dens, (pm, pn, po), (xi, yi, zi); ssh = eta + fi);

In [ ]:
@show mean(Vfluxafter[1] .^ 2)

### Verification 
If we use zero density anomamy and barotropic correction term we get right flux correction

In [ ]:
velafterb, etaafterb, Vfluxafterb =
    geostrophy(mask, 0.0 * dens, (pm, pn, po), (xi, yi, zi); ssh = fi);

In [ ]:
(Vfluxafterb[1] + Vflux[1]) ./ Vflux[1];

## Streamfunction

In [ ]:
psi = streamfunctionvolumeflux(mask, velafter, (pm, pn, po), (xi, yi, zi));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
hm = heatmap!(ax, yi[1, :, 1], -zi[1, 1, :], psi[1])
Colorbar(fig[1, 2], hm)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
hm = heatmap!(ax, xi[:, 1, 1], -zi[1, 1, :], psi[2])
Colorbar(fig[1, 2], hm)
fig